##### Copyright 2024 Google LLC.

In [17]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Read a PDF

This notebook demonstrates how you can convert a PDF file so that it can be read by the Gemini API.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/Getting_Started_with_PDF_processing.ipynb"><img src="../images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## 1. Set up Environment and create inference Client

The first task is to install the `google-genai` [Python SDK](https://googleapis.github.io/python-genai/) and obtain an API key. If you don”t have a can get one from Google AI Studio: [Get a Gemini API key](https://aistudio.google.com/app/apikey).


In [ ]:
%pip install "google-genai>=1"

Once we have the SDK and API key, we can create a client and define your model we are going to use the new Gemini 2.0 Flash model, which is available via [free tier](https://ai.google.dev/pricing#2_0flash_lite) with 1,500 request per day. 

In [17]:
from google import genai

# Comment in if you are using Colab and repace
# from google.colab import userdata
# api_key = userdata.get("GOOGLE_API_KEY")
api_key = "YOUR_API_KEY"
client = genai.Client(api_key=api_key)

model_id =  "gemini-2.0-flash" # or "gemini-2.0-flash-lite-preview-02-05"  , "gemini-2.0-pro-exp-02-05"

*Note: If you want to use Vertex AI see [here](https://googleapis.github.io/python-genai/#create-a-client) how to create your client*

## 2. Work with PDFs and other files

Gemini models are able to process [images and videos](https://ai.google.dev/gemini-api/docs/vision?lang=python#image-input), which can used with base64 strings or using the `files`api. After uploading the file we can include the file uri in the call directly. The Python API includes a [upload](https://googleapis.github.io/python-genai/#upload) and [delete](https://googleapis.github.io/python-genai/#delete) method. 

I prepared 3 PDFs one sample invoice, one invoice with scribles on it and on handwriting form. 

We can download the files to our environment using `wget`. 


In [ ]:
!wget https://huggingface.co/datasets/philschmid/pdf-samples/resolve/main/invoice.pdf
!wget https://huggingface.co/datasets/philschmid/pdf-samples/resolve/main/invoice_with_scribbles.pdf
!wget https://huggingface.co/datasets/philschmid/pdf-samples/resolve/main/handwriting.pdf

We can now upload the files using our client with the `upload` method. Let's try this for one of the files.


In [19]:
invoice_pdf = client.files.upload(file="invoice.pdf", config={'display_name': 'invoice'})

_Note: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but they cannot be downloaded. File uploads are available at no cost._

After our files are uploaded we can check to how many tokens they were converted. This not only help us understand the context we are working with it also helps to keep track of the cost. 

In [20]:
file_size = client.models.count_tokens(model=model_id,contents=invoice_pdf)
print(f'File: {invoice_pdf.display_name} equals to {file_size.total_tokens} tokens')
# File: invoice equals to 821 tokens

File: invoice equals to 821 tokens


## 3. Structured outputs with Gemini 2.0 and Pydantic

Structured Outputs is a feature that ensures Gemini always generate responses that adhere to a predefined format, such as JSON Schema. This means we have more control over the output and how to integrate it into our application as it is guaranteed to return a valid JSON object with the schema we define. 

Gemini 2.0 currenlty supports 3 dfferent types of how to define a JSON schemas:
- A single python type, as you would use in a [typing annotation](https://docs.python.org/3/library/typing.html).
- A Pydantic [BaseModel](https://docs.pydantic.dev/latest/concepts/models/)
- A dict equivalent of [genai.types.Schema](https://googleapis.github.io/python-genai/genai.html#genai.types.Schema) / [Pydantic BaseModel](https://docs.pydantic.dev/latest/concepts/models/)

Lets look at quick text-based example.

In [21]:
from pydantic import BaseModel, Field

# Define a Pydantic model
# We use the Field class to add a description and default value to provide more context to the model
class Topic(BaseModel):
    name: str = Field(description="The name of the topic")

class Person(BaseModel):
    first_name: str = Field(description="The first name of the person")
    last_name: str = Field(description="The last name of the person")
    age: int = Field(description="The age of the person, if not provided please return 0")
    work_topics: list[Topic] = Field(description="The fields of interest of the person, if not provided please return an empty list")


# Define the prompt
prompt = "Philipp Schmid is a Senior AI Developer Relations Engineer at Google DeepMind working on Gemini, Gemma with the mission to help every developer to build and benefit from AI in a responsible way.  "

# Generate a response using the Person model
response = client.models.generate_content(model=model_id, contents=prompt, config={'response_mime_type': 'application/json',
        'response_schema': Person})

# print the response as a json string
print(response.text)

# sdk automatically converts the response to the pydantic model
philipp: Person = response.parsed

# access an attribute of the json response
print(f"First name is {philipp.first_name}")

{
  "age": 0,
  "first_name": "Philipp",
  "last_name": "Schmid",
  "work_topics": [
    {
      "name": "AI"
    },
    {
      "name": "Gemini"
    },
    {
      "name": "Gemma"
    },
    {
      "name": "Responsible AI"
    },
    {
      "name": "Developer Relations"
    }
  ]
}
First name is Philipp


## 4. Extract Structured data from PDFs using Gemini 2.0

Now, let's combine the File API and structured output to extract information from our PDFs. We can create a simple method that accepts a local file path and a pydantic model and return the structured data for us. The method will:

1. Upload the file to the File API
2. Generate a structured response using the Gemini API
3. Convert the response to the pydantic model and return it


In [22]:
def extract_structured_data(file_path: str, model: BaseModel):
    # Upload the file to the File API
    file = client.files.upload(file=file_path, config={'display_name': file_path.split('/')[-1].split('.')[0]})
    # Generate a structured response using the Gemini API
    prompt = f"Extract the structured data from the following PDF file"
    response = client.models.generate_content(model=model_id, contents=[prompt, file], config={'response_mime_type': 'application/json', 'response_schema': model})
    # Convert the response to the pydantic model and return it
    return response.parsed

In our Example every PDF is a different to each other. So we want to define unique Pydantic models for each PDF to show the performance of the Gemini 2.0. If you have very similar PDFs and want to extract the same information you can use the same model for all of them. 

We are going to:
- `Invoice.pdf` : Extract the invoice number, date and all list items with description, quantity and gross worth and the total gross worth
- `Invoice_with_scribles.pdf` : Extract the total balance
- `handwriting.pdf` : Extract the form number, fiscal start date, fiscal end date, and the plan liabilities beginning of the year and end of the year

You can inspect the PDFs using a local PDF viewer or in the [browser](https://huggingface.co/datasets/philschmid/pdf-samples/tree/main).

_Note: Using Pydantic features we can add more context to the model to make it more accurate as well as some validation to the data. Adding a comprehensive description can significantly improve the performance of the model. Libraries like [instructor](https://python.useinstructor.com/) added automatic retries based on validation errors, which can be a great help, but come at the cost of additional requests._


### Invoice.pdf

![Invoice.pdf](https://huggingface.co/datasets/philschmid/pdf-samples/resolve/main/invoice.png)

In [24]:
from pydantic import BaseModel, Field

class Item(BaseModel):
    description: str = Field(description="The description of the item")
    quantity: float = Field(description="The Qty of the item")
    gross_worth: float = Field(description="The gross worth of the item")

class Invoice(BaseModel):
    """Extract the invoice number, date and all list items with description, quantity and gross worth and the total gross worth."""
    invoice_number: str = Field(description="The invoice number e.g. 1234567890")
    date: str = Field(description="The date of the invoice e.g. 2024-01-01")
    items: list[Item] = Field(description="The list of items with description, quantity and gross worth")
    total_gross_worth: float = Field(description="The total gross worth of the invoice")


result = extract_structured_data("invoice.pdf", Invoice)
print(type(result))
print(f"Extracted Invoice: {result.invoice_number} on {result.date} with total gross worth {result.total_gross_worth}")
for item in result.items:
    print(f"Item: {item.description} with quantity {item.quantity} and gross worth {item.gross_worth}")

<class '__main__.Invoice'>
Extracted Invoice: 27301261 on 10/09/2012 with total gross worth 544.46
Item: Lilly Pulitzer dress Size 2 with quantity 5.0 and gross worth 247.5
Item: New ERIN Erin Fertherston Straight Dress White Sequence Lining Sleeveless SZ 10 with quantity 1.0 and gross worth 65.99
Item: Sequence dress Size Small with quantity 3.0 and gross worth 115.5
Item: fire los angeles dress Medium with quantity 3.0 and gross worth 21.45
Item: Eileen Fisher Women's Long Sleeve Fleece Lined Front Pockets Dress XS Gray with quantity 3.0 and gross worth 52.77
Item: Lularoe Nicole Dress Size Small Light Solid Grey/ White Ringer Tee Trim with quantity 2.0 and gross worth 8.25
Item: J.Crew Collection Black & White sweater Dress sz S with quantity 1.0 and gross worth 33.0


Fantastic! If we compare this to the original invoice we can see that the information is extracted correctly. 


### Invoice_with_scribbles.pdf

![Invoice_with_scribbles.pdf](https://huggingface.co/datasets/philschmid/pdf-samples/resolve/main/invoice_with_scribbles.png)

In [27]:
class InvoiceWithScribles(BaseModel):
    total_balance: float = Field(description="The total balance of the invoice")

result = extract_structured_data("invoice_with_scribbles.pdf", InvoiceWithScribles)
print(type(result))
print(f"Extracted Invoice: {result.total_balance}")

<class '__main__.InvoiceWithScribles'>
Extracted Invoice: 1350.0


Awesome, Gemini didn't get distracted by the scribles and extracted the total balance correctly. 


### handwriting.pdf

![handwriting.pdf](https://huggingface.co/datasets/philschmid/pdf-samples/resolve/main/handwriting.png)

In [29]:
class handwriting(BaseModel):
    """Extract the form number, fiscal start date, fiscal end date, and the plan liabilities beginning of the year and end of the year."""
    form_number: str = Field(description="The Form Number")
    fiscal_start_date: str = Field(description="The fiscal start date of the handwriting")
    fiscal_end_date: str = Field(description="The fiscal end date of the handwriting")
    plan_liabilities_beginning_of_year: float = Field(description="The plan liabilities beginning of the year")
    plan_liabilities_end_of_year: float = Field(description="The plan liabilities end of the year")

result = extract_structured_data("handwriting.pdf", handwriting)

print(f'Extracted Form Number: {result.form_number} with fiscal start date {result.fiscal_start_date} and fiscal end date {result.fiscal_end_date}. \nPlan liabilities beginning of the year {result.plan_liabilities_beginning_of_year} and end of the year {result.plan_liabilities_end_of_year}')

Extracted Form Number: CA530082 with fiscal start date 01/02/2022 and fiscal end date 01/02/2023. 
Plan liabilities beginning of the year 4000.0 and end of the year 5000.0
